In [ ]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
import nltk
from contractions import contractions_dict
from nltk.corpus import stopwords
from spacy.lang.en.stop_words import STOP_WORDS
from itertools import filterfalse
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report
import spacy

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
data = pd.read_csv("/content/emails.csv")

In [ ]:
data

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1
...,...,...
5723,Subject: re : research and development charges...,0
5724,"Subject: re : receipts from visit jim , than...",0
5725,Subject: re : enron case study update wow ! a...,0
5726,"Subject: re : interest david , please , call...",0


In [ ]:
def strip_titles(text):
    if "Subject: re :" in text:
        return text[13:]
    elif "Subject: news :" in text:
        return text[15:]
    else:
        return text[8:]

In [ ]:
data['text'] = data['text'].apply(lambda x: strip_titles(x))

In [ ]:
data['text'] = data['text'].apply(lambda x: word_tokenize(x))

In [ ]:
def normalize_tokens(list_of_tokens):
    return map(lambda x: x.lower(),list_of_tokens)

In [ ]:
data['text'] = data['text'].apply(lambda x: normalize_tokens(x))

In [ ]:
data['text'] = data['text'].apply(lambda x: list(x))

In [ ]:
def contracted_word_expansion(token):
    if token in contractions_dict.keys():
        return contractions_dict[token]
    else:
        return token

In [ ]:
def contractions_expansion(list_of_tokens):
    return map(contracted_word_expansion,list_of_tokens)

In [ ]:
data['text'] = data['text'].apply(lambda x: contractions_expansion(x))

In [ ]:
data['text'] = data['text'].apply(lambda x: list(x))

In [ ]:
regex = r'^@[a-zA-z0-9]|^#[a-zA-Z0-9]|\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*|\W+|\d+|<("[^"]*"|\'[^\']*\'|[^\'">])*>|_+|[^\u0000-\u007f]+'

In [ ]:
def waste_word_or_not(token):
    return re.search(regex,token)

In [ ]:
def filter_waste_words(list_of_tokens):
    return filterfalse(waste_word_or_not,list_of_tokens)

In [ ]:
data['text'] = data['text'].apply(lambda x: filter_waste_words(x))

In [ ]:
data['text'] = data['text'].apply(lambda x: list(x))

In [ ]:
def split(list_of_tokens):
    return map(lambda x: re.split(regex,x)[0],list_of_tokens)

In [ ]:
data['text'] = data['text'].apply(lambda x: split(x))

In [ ]:
data['text'] = data['text'].apply(lambda x: list(x))

In [ ]:
en_stop_words = list(set(stopwords.words('english')).union(set(STOP_WORDS)))

In [ ]:
def is_stopword(token):
    return not(token in en_stop_words or re.search(r'\b\w\b|[^\u0000-\u007f]+|_+|\W+',token))

In [ ]:
def stopwords_removal(list_of_tokens):
    return filter(is_stopword,list_of_tokens)

In [ ]:
data['text'] = data['text'].apply(lambda x: stopwords_removal(x))

In [ ]:
data['text'] = data['text'].apply(lambda x: list(x))

In [ ]:
def get_wnet_pos_tag(treebank_tag):
    if treebank_tag[1].startswith('J'):
        return (treebank_tag[0],wordnet.ADJ)
    elif treebank_tag[1].startswith('V'):
        return (treebank_tag[0],wordnet.VERB)
    elif treebank_tag[1].startswith('N'):
        return (treebank_tag[0],wordnet.NOUN)
    elif treebank_tag[1].startswith('R'):
        return (treebank_tag[0],wordnet.ADV)
    else:
        (treebank_tag[0],wordnet.NOUN)

In [ ]:
def get_pos_tag(list_of_tokens):
    return map(get_wnet_pos_tag,pos_tag(list_of_tokens))

In [ ]:
data['text'] = data['text'].apply(lambda x: get_pos_tag(x))

In [ ]:
data['text'] = data['text'].apply(lambda x: list(x))

In [ ]:

lemmatizer = WordNetLemmatizer()

In [ ]:
def token_lemmatization(token_pos_tuple):
    if token_pos_tuple == None:
        return ""
    else:
        return lemmatizer.lemmatize(word=token_pos_tuple[0],pos=token_pos_tuple[1])

In [ ]:

def lemmatization(list_of_tokens):
    if len(list_of_tokens) > 0:
        return map(lambda x: token_lemmatization(x),list_of_tokens)

In [ ]:
data['text'] = data['text'].apply(lambda x: lemmatization(x))

In [ ]:
data['text'] = data['text'].apply(lambda x: list(x))

In [ ]:
vocab = set()
for list_of_tokens in data['text']:
    vocab = vocab.union(set(list_of_tokens))

In [ ]:
vocab = list(vocab)

In [ ]:
vocab.pop(0)

''

In [ ]:
vocab_dict = dict(zip(vocab,list(range(0,len(vocab)))))

In [ ]:
def join_tokens(list_of_tokens):
    return " ".join(list_of_tokens)

In [ ]:
data['text'] = data['text'].apply(lambda x: join_tokens(x))

In [ ]:
corpus = list()
for email_text in data['text']:
    corpus.append(email_text)

In [ ]:
vectorizer = TfidfVectorizer(vocabulary=vocab_dict)
tf_idf_matrix = vectorizer.fit_transform(corpus)

In [ ]:
tf_idf_matrix = tf_idf_matrix.toarray()

In [ ]:
df = pd.DataFrame(tf_idf_matrix)

In [ ]:
df['spam'] = data['spam']

In [ ]:
pca = PCA(n_components=800)

In [ ]:
tf_idf_matrix_reduced = pca.fit_transform(tf_idf_matrix)

In [ ]:
df = pd.DataFrame(data=tf_idf_matrix_reduced)

In [ ]:
df['spam'] = data['spam']

In [ ]:
gnb = GaussianNB()

In [ ]:
X_train = df.iloc[:,0:800]
y_train = df['spam']

In [ ]:
gnb.fit(X=X_train,y=y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [ ]:
predicted_categories = gnb.predict(X_train)

In [ ]:
print(classification_report(y_true=y_train,y_pred=predicted_categories))

              precision    recall  f1-score   support

           0       0.90      0.91      0.91      4360
           1       0.71      0.69      0.70      1368

    accuracy                           0.86      5728
   macro avg       0.81      0.80      0.80      5728
weighted avg       0.86      0.86      0.86      5728

